In [46]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [47]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.layers import Input,Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.densenet import DenseNet169

In [48]:
def generate_dataset_from_directory(folder_path, size= 224, batch_size=32, seed=123):
    image_generator=ImageDataGenerator(
    samplewise_center=True,
    samplewise_std_normalization=True,
    validation_split=0.3)
    
    train_data=image_generator.flow_from_directory(
    directory=folder_path,
    class_mode='categorical',
    target_size=(size,size),
    batch_size=batch_size,
    shuffle=True,
    seed=seed,
    subset='training'
    )
    
    val_data=image_generator.flow_from_directory(
    directory=folder_path,
    class_mode='categorical',
    target_size=(size,size),
    batch_size=batch_size,
    shuffle=True,
    seed=seed,
    subset='validation'
    )
    return train_data, val_data

In [49]:
folder_path='../input/covid19-radiography-database/COVID-19_Radiography_Dataset'
train_data,val_data= generate_dataset_from_directory(folder_path,size=128)

In [50]:
class_names=train_data.class_indices
print(class_names)

In [51]:
classes=list(class_names.keys())
print(classes)

In [52]:
images, labels = next(train_data)
labels= np.argmax(labels, axis=1)
class_dict= train_data.class_indices
class_dict_inv= dict((v,k) for k, v in class_dict.items())
y_names= [class_dict_inv[key] for key in labels]

In [53]:
plt.figure()
for image in images:
    j=0
    for i in range(4):
        ax= plt.subplot(1,4,i+1)
        for k in range(len(labels)):
            if labels[k]==j:
                plt.imshow(images[k])
                plt.title(classes[i])
                plt.axis('off')
                break
        j+=1

In [54]:
raw_image= images[k]
plt.imshow(raw_image, cmap='gray')
plt.colorbar()
plt.title('Raw Chest X Ray Image')

In [63]:
def plotLearningCurve(history, epoch):
    epochRange = range(1,epoch+1)
    fig, ax= plt.subplots(1,2, figsize=(20,10))
    ax[0].plot(epochRange, history.history['accuracy'],'b',label='Training Accuracy')
    ax[0].plot(epochRange, history.history['val_accuracy'],'r',label='Validation Accuracy')
    ax[0].set_title('Training and Validation Accuracy')
    ax[0].set_xlabel('Epoch', fontsize=20)
    ax[0].set_ylabel('Accuracy', fontsize=20)
    ax[0].legend()
    ax[0].grid(color='gray', linestyle='--')
    ax[1].plot(epochRange, history.history['loss'],'b',label='Training Loss')
    ax[1].plot(epochRange, history.history['val_loss'],'r',label='Validation Loss')
    ax[1].set_title('Training and Validation Loss')
    ax[1].set_xlabel('Epoch', fontsize=20)
    ax[1].set_ylabel('Loss', fontsize=20)
    ax[1].legend()
    ax[1].grid(color='gray', linestyle='--')
    plt.show()

In [56]:
def cnn_model(train_data,test_data, epochs,size):
#Building the model using Keras functional API
    print("----Building the model----")
    i = Input(shape=(size,size,3)) #defining input
    x = BatchNormalization()(i)
    x = Conv2D(32,3, activation= 'relu')(i) #adding convolution layers
    x = MaxPooling2D()(x)
    x = Dropout(0.2)(x)
    x = Conv2D(64, 3, activation='relu')(x)
    x = MaxPooling2D()(x)
    x = Dropout(0.2)(x)
    x = Conv2D(128, 3, activation='relu')(x)
    x = MaxPooling2D()(x)
    x = Dropout(0.2)(x)
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(4, activation= 'softmax')(x)

    model = Model(i,x)
    model.summary()
#Training the Convolutional Neural Network
    print("----Training the network----")
    model.compile(optimizer= Adam(0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    
    early_stop = EarlyStopping(monitor='val_loss', 
                               mode='min', 
                               patience = 5 ,
                               restore_best_weights=True)
    #model_checkpoint
    mc = ModelCheckpoint('cnn_model.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
    r = model.fit(train_data, 
                  validation_data = val_data, 
                  epochs = epochs, 
                  verbose = 2,
                  batch_size = 32,
                  callbacks = [early_stop,mc])
    print("Train score:", model.evaluate(train_data))
    print("Test score:", model.evaluate(val_data))
    n_epochs = len(r.history['loss'])
    
    return r,model,n_epochs 

In [58]:
epochs = 20
r, model, n_epochs = cnn_model(train_data, val_data, epochs, 128)

In [64]:
plotLearningCurve(r,n_epochs)

In [65]:
# printing model accuracy for train and test data
test_evaluation = model.evaluate(val_data)
print(f"Test Accuracy using CNN: {test_evaluation[1] * 100:.2f}%")

train_evaluation = model.evaluate(train_data)
print(f"Train Accuracy using CNN: {train_evaluation[1] * 100:.2f}%")